### 필요한 라이브러리

In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from collections import defaultdict
import csv

### COSPI 200 편입 종목 크롤링

In [8]:
BaseUrl = 'http://finance.naver.com/sise/entryJongmok.nhn?&page='

# 종목 코드 추출
stock_codes = []

for i in range(1, 21):  # range() 함수의 끝 파라미터는 포함되지 않는다.
    url = BaseUrl + str(i)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    items = soup.find_all('td', {'class': 'ctg'})
    for item in items:
        txt = item.a.get('href')  # 예: https://finance.naver.com/item/main.nhn?code=006390
        k = re.search('[\d]+', txt)  # 정규표현식 사용. [\d] 숫자표현, + : 반복
        if k:
            code = k.group()
            stock_codes.append(code)

print(stock_codes)

<>:13: SyntaxWarning: invalid escape sequence '\d'
<>:13: SyntaxWarning: invalid escape sequence '\d'
/var/folders/vw/vspwd_yn6b94yd37dg899j380000gn/T/ipykernel_18957/155905241.py:13: SyntaxWarning: invalid escape sequence '\d'
  k = re.search('[\d]+', txt)  # 정규표현식 사용. [\d] 숫자표현, + : 반복


['005930', '000660', '373220', '207940', '005380', '012450', '068270', '000270', '329180', '105560', '035420', '055550', '012330', '042660', '138040', '005490', '028260', '009540', '034020', '086790', '259960', '000810', '032830', '035720', '010130', '015760', '011200', '051910', '096770', '033780', '316140', '030200', '010140', '064350', '024110', '006400', '402340', '066570', '017670', '352820', '267260', '323410', '003550', '018260', '003670', '034730', '000100', '009150', '047050', '086280', '326030', '047810', '003490', '272210', '042700', '003230', '090430', '079550', '006800', '010620', '005830', '021240', '267250', '010120', '010950', '180640', '051900', '032640', '009830', '161390', '005940', '000150', '271560', '029780', '241560', '016360', '000720', '071050', '298040', '377300', '034220', '006260', '450080', '011790', '251270', '028050', '022100', '001040', '000880', '097950', '078930', '036460', '035250', '128940', '039490', '175330', '011070', '138930', '004020', '454910',

### 편입 종목을 같은 업종코드(산업군)끼리 분류 후 클러스터로 묶음

In [ ]:
# 헤더 설정
headers = {"User-Agent": "Mozilla/5.0"}

# 업종코드를 추출하는 함수
def get_upjong_code_by_stock(stock_code):
    url = f"https://finance.naver.com/item/main.naver?code={stock_code}"
    try:
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.text, "html.parser")
        upjong_link = soup.find("a", href=lambda href: href and "sise_group_detail.naver?type=upjong" in href)
        if upjong_link:
            href = upjong_link["href"]
            upjong_code = href.split("no=")[-1]
            return upjong_code
    except Exception as e:
        print(f"[오류] {stock_code} 처리 중 예외 발생: {e}")
    return None

# 업종코드별 종목코드 모으기
upjong_groups = defaultdict(list)

for i, code in enumerate(stock_codes):
    upjong_code = get_upjong_code_by_stock(code)
    if upjong_code:
        upjong_groups[upjong_code].append(code)
        print(f"[{i+1}/{len(stock_codes)}] 종목 {code} → 업종코드 {upjong_code}")
    else:
        print(f"[{i+1}/{len(stock_codes)}] 종목 {code} → 업종코드 없음")
    time.sleep(0.3)

# 업종코드별로 '클러스터 ID'를 부여함
upjong_to_cluster = {upjong: idx for idx, upjong in enumerate(upjong_groups.keys())}

# 결과를 리스트로 변환함
result = []
for upjong_code, codes in upjong_groups.items():
    cluster_id = upjong_to_cluster[upjong_code]
    for code in codes:
        result.append({
            "종목코드": code,
            "업종코드": upjong_code,
            "Cluster": cluster_id
        })

# 데이터프레임으로 보기 좋게 출력함
df = pd.DataFrame(result)
print("\n📦 최종 결과:")
print(df)

[1/200] 종목 005930 → 업종코드 278
[2/200] 종목 000660 → 업종코드 278
[3/200] 종목 373220 → 업종코드 283
[4/200] 종목 207940 → 업종코드 261
[5/200] 종목 005380 → 업종코드 273
[6/200] 종목 012450 → 업종코드 284
[7/200] 종목 068270 → 업종코드 261
[8/200] 종목 000270 → 업종코드 273
[9/200] 종목 329180 → 업종코드 291
[10/200] 종목 105560 → 업종코드 301
[11/200] 종목 035420 → 업종코드 300
[12/200] 종목 055550 → 업종코드 301
[13/200] 종목 012330 → 업종코드 270
[14/200] 종목 042660 → 업종코드 291
[15/200] 종목 138040 → 업종코드 321
[16/200] 종목 005490 → 업종코드 304
[17/200] 종목 028260 → 업종코드 276
[18/200] 종목 009540 → 업종코드 291
[19/200] 종목 034020 → 업종코드 299
[20/200] 종목 086790 → 업종코드 301
[21/200] 종목 259960 → 업종코드 263
[22/200] 종목 000810 → 업종코드 315
[23/200] 종목 032830 → 업종코드 330
[24/200] 종목 035720 → 업종코드 300
[25/200] 종목 010130 → 업종코드 322
[26/200] 종목 015760 → 업종코드 325
[27/200] 종목 011200 → 업종코드 323
[28/200] 종목 051910 → 업종코드 272
[29/200] 종목 096770 → 업종코드 313
[30/200] 종목 033780 → 업종코드 275
[31/200] 종목 316140 → 업종코드 301
[32/200] 종목 030200 → 업종코드 336
[33/200] 종목 010140 → 업종코드 291
[34/200] 종목 064350 

In [ ]:
# 클러스터 개수와 종목 확인
df

,종목코드,업종코드,Cluster
0,005930,278,0
1,000660,278,0
2,042700,278,0
3,373220,283,1
4,006400,283,1
...,...,...,...
195,137310,281,47
196,145720,281,47
197,009240,303,48
198,014820,311,49


### CSV로 저장

In [18]:
# 종목코드 앞에 0이오면 0이 사라진 후 저장되는 문제가 발생
# 그 이유는 종목코드를 int로 인지했기 때문. 그래서 문자열로 바꿔줌
# 000660 같은 경우, 660으로 저장되었었음

df['종목코드'] = df['종목코드'].astype(str)
df.to_csv("종목_업종_클러스터.csv", index=False)